## Sparse Retrieval
Implementation of sparse passage retrieval using TF-IDF and BM25. Evaluated on the MS MARCO dataset using MRR and retrieval time.

## Load Dataset

In [1]:
from dataset_loader import DatasetLoader

# Load cranfield dataset for testing functions, use MS MARCO, HotpotQA, and potentially Climate-FEVER for real evaluation
loader = DatasetLoader("cranfield")
docs, queries, qrels = loader.get_all()
loader.print_info()

# # load multiple datasets for evaluation
# nameset = ["beir/msmarco/dev", "bier/hotpotqa/dev", "bier/climate-fever/dev"]
#
# # Dictionary to hold datasets
# datasets = {}
#
# for name in nameset:
#     loader = DatasetLoader(name)
#     docs, queries, qrels = loader.get_all()
#     datasets[name] = {
#         "docs": docs,
#         "queries": queries,
#         "qrels": qrels
#     }
#     loader.print_info()


DATASET: cranfield
DOCS (1400): ('1', 'experimental investigation of the aerodynamics of a\nwing in a slipstream .\n  an experimental study of a wing in a propeller slipstream was\nmade in order to determine the spanwise distribution of the lift\nincrease due to slipstream at different angles of attack of the wing\nand at different free stream to slipstream velocity ratios .  the\nresults were intended in part as an evaluation basis for different\ntheoretical treatments of this problem .\n  the comparative span loading curves, together with\nsupporting evidence, showed that a substantial part of the lift increment\nproduced by the slipstream was due to a /destalling/ or\nboundary-layer-control effect .  the integrated remaining lift\nincrement, after subtracting this destalling lift, was found to agree\nwell with a potential flow theory .\n  an empirical evaluation of the destalling effects was made for\nthe specific configuration of the experiment .') 

QUERIES (225): ('1', 'what simi

## Data Preprocessing

In [2]:
# extract document and query IDs + texts
doc_ids, doc_texts = list(docs.keys()), list(docs.values())
query_ids, query_texts = list(queries.keys()), list(queries.values())

## Build Retriever

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss
from rank_bm25 import BM25Okapi

# TF-IDF
tfidf = TfidfVectorizer(lowercase=True, stop_words="english")
doc_tfidf = tfidf.fit_transform(doc_texts)
query_tfidf = tfidf.transform(query_texts)

# convert to array for FAISS
doc_tfidf_arr = doc_tfidf.toarray().astype('float32')
query_tfidf_arr = query_tfidf.toarray().astype('float32')

# normalize
faiss.normalize_L2(doc_tfidf_arr)
faiss.normalize_L2(query_tfidf_arr)

# build index
dim = doc_tfidf_arr.shape[1]
index = faiss.IndexFlatIP(dim)  # Inner product (cosine similarity)
index.add(doc_tfidf_arr)  # add documents to index
# print(index.ntotal)

# BM25
# Use the same preprocessor and tokenizer as TF-IDF
tokenized_docs = [tfidf.build_tokenizer()(tfidf.build_preprocessor()(doc)) for doc in doc_texts]
tokenized_queries = [tfidf.build_tokenizer()(tfidf.build_preprocessor()(query)) for query in query_texts]

bm25 = BM25Okapi(tokenized_docs)


## Evaluation

### Retrieval Time

In [4]:
import time
from heapq import nlargest

k = 10

start_time = time.time()
distances, indices = index.search(query_tfidf_arr, k)  # search for top k documents
retrieval_time = (time.time() - start_time) / len(query_ids)
print(f"TF-IDF Retrieval time per query: {retrieval_time:.6f} seconds")

# store results in a dictionary
tfidf_scores = {
    query_ids[i]: {
        doc_ids[indices[i][j]]: float(distances[i][j]) for j in range(k)
    }
    for i in range(len(query_ids))
}

# Score each query against the corpus
bm25_scores = {}
bm25_total_time = 0  # total time for all queries

for query_id, query_tokens in zip(query_ids, tokenized_queries):
    start_time = time.time()
    scores = bm25.get_scores(query_tokens) # get scores for all documents
    bm25_total_time += time.time() - start_time

    # Get top-k indices sorted by score
    top_k = nlargest(k, enumerate(scores), key=lambda x: x[1])

    bm25_scores[query_id] = {
        doc_ids[i]: float(score)
        for i, score in top_k
    }

# Average retrieval time per query
bm25_time = bm25_total_time / len(query_ids)
print(f"BM25 Retrieval time per query: {bm25_time:.6f} seconds")

TF-IDF Retrieval time per query: 0.000048 seconds
BM25 Retrieval time per query: 0.002192 seconds


### Mean Reciprocal Rank (MRR)

In [5]:
from ranx import Run, evaluate

# calculate MRR
# Run: stores the relevance scores estimated by the model under evaluation
tfidf_run = Run.from_dict(tfidf_scores, name="tfidf")
bm25_run = Run.from_dict(bm25_scores, name="bm25")

# save results to file
tfidf_run.save("tfidf_run.json")
bm25_run.save("bm25_run.json")

tfidf_mrr = evaluate(qrels, tfidf_run, "mrr@10", make_comparable=True)
bm25_mrr = evaluate(qrels, bm25_run, "mrr@10", make_comparable=True)
print(f"TF-IDF MRR: {tfidf_mrr:.4f}")
print(f"BM25 MRR: {bm25_mrr:.4f}")

TF-IDF MRR: 0.4920
BM25 MRR: 0.4928


In [6]:
from itertools import islice

def print_topk_comparison(tfidf_scores, bm25_scores, qrels, k=10, num_queries=5):
    """
    Print the top-k documents for each query from both TF-IDF and BM25 scores. Correct relevant documents will be wrapped in *stars*.
    """
    print(f"{'Query ID':<10} | {'TF-IDF Top-K':<40} | {'BM25 Top-K':<40}")
    print("-" * 100)

    for query_id in islice(tfidf_scores.keys(), num_queries):
        tfidf_top = sorted(tfidf_scores[query_id].items(), key=lambda x: x[1], reverse=True)[:k]
        bm25_top = sorted(bm25_scores[query_id].items(), key=lambda x: x[1], reverse=True)[:k]

        tfidf_docs = [f"*{doc_id}*" if doc_id in qrels.get(query_id, {}) and qrels[query_id][doc_id] > 0 else doc_id
                      for doc_id, _ in tfidf_top]
        bm25_docs = [f"*{doc_id}*" if doc_id in qrels.get(query_id, {}) and qrels[query_id][doc_id] > 0 else doc_id
                     for doc_id, _ in bm25_top]

        print(f"{query_id:<10} | {', '.join(tfidf_docs):<40} | {', '.join(bm25_docs):<40}")


# Run comparison
qrels_dict = qrels.to_dict()
print_topk_comparison(tfidf_scores, bm25_scores, qrels_dict, k=5, num_queries=5)


Query ID   | TF-IDF Top-K                             | BM25 Top-K                              
----------------------------------------------------------------------------------------------------
1          | *13*, *184*, *12*, *51*, 486             | *184*, 486, *13*, *12*, 1268            
2          | *12*, *51*, *746*, 884, 1169             | *12*, *746*, 792, *14*, 1089            
3          | *5*, 485, *399*, *181*, *144*            | *5*, *399*, *181*, *144*, 485           
4          | *166*, 1275, 488, 1189, *236*            | *166*, 1189, 488, 1061, 185             
5          | 103, 746, 1272, 1379, 410                | 103, 1032, 943, *1296*, 746             
